In [16]:
import sys
import json
import requests
import time
import argparse
import pandas as pd
import numpy as np
import pymongo

import random

import boto3
import boto.s3.connection
from boto.s3.key import Key
from src.Airbnb_Scraping_tools import *
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.request import urlretrieve 

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load in data

In [18]:
s3_path = 'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [19]:
s3_path

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [20]:
folder = 'Boston-Massachusetts-US'
file_tmp = pd.read_csv(s3_path+folder+'/listings.csv.gz',compression = 'gzip')
print('Total number of apartments in ' + folder +" is: {}".format(len(file_tmp)))
file_tmp.describe()

Total number of apartments in Boston-Massachusetts-US is: 4870


,id,scrape_id,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,calculated_host_listings_count,reviews_per_month
count,4.870000e+03,4.870000e+03,4.870000e+03,0.0,4870.000000,4870.000000,0.0,4870.000000,4870.000000,4870.000000,...,3936.000000,3941.000000,3928.000000,3940.000000,3927.000000,3927.000000,0.0,0.0,4870.000000,3986.000000
mean,1.293651e+07,2.017101e+13,4.088556e+07,NaN,78.667351,78.667351,NaN,42.339999,-71.083943,3.298152,...,9.588923,9.443035,9.742617,9.713706,9.487395,9.334861,NaN,NaN,19.301848,2.204792
std,6.202199e+06,5.000513e-01,3.971610e+07,NaN,219.284219,219.284219,NaN,0.025338,0.032768,2.131683,...,0.773592,0.957797,0.685026,0.722654,0.822990,0.890983,NaN,NaN,43.029043,2.336672
min,3.781000e+03,2.017101e+13,4.240000e+03,NaN,0.000000,0.000000,NaN,42.235760,-71.179257,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,1.000000,0.020000
25%,7.913572e+06,2.017101e+13,1.011587e+07,NaN,1.000000,1.000000,NaN,42.329070,-71.105007,2.000000,...,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,NaN,1.000000,0.450000
50%,1.431376e+07,2.017101e+13,2.695608e+07,NaN,2.000000,2.000000,NaN,42.345512,-71.077124,2.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,NaN,NaN,2.000000,1.300000
75%,1.857908e+07,2.017101e+13,6.297457e+07,NaN,11.000000,11.000000,NaN,42.356009,-71.061114,4.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,9.000000,3.247500
max,2.123117e+07,2.017101e+13,1.533133e+08,NaN,982.000000,982.000000,NaN,42.396191,-70.985047,16.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,181.000000,15.100000


# Open S3 bucket

In [7]:
bucket_name = "chen-gal-test"

In [36]:
s3 = boto3.client("s3")

remote_pathname_l1 = 'AirbnbData/'
remote_pathname_l2 = folder
remote_pathname = remote_pathname_l1 + remote_pathname_l2

# s3.put_object(Body=file , Bucket=bucket_name, Key=remote_pathname)


In [50]:
logfile_name = 'DataSet/'+'logfile_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')




# Downloading

In [ ]:
apt_infos = {}
apt_jses = {}

In [52]:
# for i in range(len(file_tmp)):
for i in range(1):
    print(i)
    url = file_tmp.loc[i,'listing_url']
    apt_id = file_tmp.loc[i,'id']
    apt_name = file_tmp.loc[i,'name']
    with open(logfile_name,'w+') as f:
        f.write(f'---------------------\nApartment{i} | {apt_id} | {url} \n')         
    try:
        return_parsed = parse_each_webpage(url)
    except CannotOpenUrl:        
        with open(logfile_name,'a') as f:
            f.writelines('Cannot open '+ str(i)+ ' url.\t' + 'Id: '+ str(apt_id) + ' |  Url: '+ url)                        
        print('Failed to get access to URL, sleep for 20 secs ')
        time.sleep(20)
        return_parsed = parse_each_webpage(url)
    finally:
        scrape_info,ori_json = return_parsed[0],return_parsed[1]
        if len(scrape_info) == 0:
            with open(logfile_name,'a') as f:
                f.writelines('Failed to scrape the url, simply save the json file\n')
        else:
            with open(logfile_name,'a') as f:
                f.writelines('Successfully scraped the url, save both files\n')
            
        dict_apt = {'apt_name':apt_name,'url' : url, 'info': scrape_info}
        apt_jses.update({str(apt_id):ori_json})
        apt_infos.update({str(apt_id):dict_apt})
        with open(logfile_name,'a') as f:
                f.writelines(f'Finished scraping {i} apartment! ID:{apt_id}\n')
        if i % 100 == 0 and i != 0:
            
            js_filename = remote_pathname +"/jsons/ori_json_"+str(i)            
            s3.put_object(Body=json.dump(apt_jses) , Bucket=bucket_name, Key=js_filename)
            
            print("Wrote  down"+str(i)+"json file")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved json file {filename}!\n')
            
            jses = {}
        time.sleep(random.random()+2)
        
#upload log_file to s3
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)


0
request_url:https://www.airbnb.com/rooms/20872145


## Save the result

### save scrapted data

In [58]:
scraped_filename = remote_pathname +"/ws_data/webscrapted.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

{'ETag': '"a365eb8605abed64c3037e75e4d3c608"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 00:21:06 GMT',
   'etag': '"a365eb8605abed64c3037e75e4d3c608"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'T74AOyf6fSqedrpK+eJK4AKbVuFFBjU0o9/lq6NDWorDY3maz95ZP+XRPy8gxdiYILcy+g0cajU=',
   'x-amz-request-id': '5C9A18202FFC0F15'},
  'HTTPStatusCode': 200,
  'HostId': 'T74AOyf6fSqedrpK+eJK4AKbVuFFBjU0o9/lq6NDWorDY3maz95ZP+XRPy8gxdiYILcy+g0cajU=',
  'RequestId': '5C9A18202FFC0F15',
  'RetryAttempts': 0}}

### save json data

In [60]:
last_piece_js_filename = remote_pathname+"/ws_data/ori_json_lastpiece.json"

s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=last_piece_js_filename)


{'ETag': '"da6952ffc3ef10a8a01d8eb35bbb70b3"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 00:22:01 GMT',
   'etag': '"da6952ffc3ef10a8a01d8eb35bbb70b3"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'RM0h99zmfC0YbGI6pvpg16Iw3xTZe8eCz8etpLxALWVUDnYF11BNmOuH2BZObwYhUeCpqS6wBOM=',
   'x-amz-request-id': '453D9101B5D3F13C'},
  'HTTPStatusCode': 200,
  'HostId': 'RM0h99zmfC0YbGI6pvpg16Iw3xTZe8eCz8etpLxALWVUDnYF11BNmOuH2BZObwYhUeCpqS6wBOM=',
  'RequestId': '453D9101B5D3F13C',
  'RetryAttempts': 0}}

## Load json data from s3

In [69]:
response = s3.get_object(Bucket=bucket_name,
                         Key=scraped_filename)

In [70]:
scraped_data = json.loads(response['Body'].read())